In [1]:
import os
import json

In [11]:
def flatten(l):
    fl = [item for sublist in l for item in sublist]
    return fl

In [2]:

datasets_path = "/Users/sophierand/RCDatasets/datasets.json"
with open(datasets_path) as json_file:
    datasets = json.load(json_file)


In [54]:
ncses_datasets = [{'id':d['id'],'title':list((d['title'],d['alt_title'][0]))} for d in datasets if d['id'] in ['dataset-370','dataset-371']]
ncses_datasets

[{'id': 'dataset-370', 'title': ['Survey of Earned Doctorates', 'SED']},
 {'id': 'dataset-371', 'title': ['Survey of Doctorate Recipients', 'SDR']}]

In [19]:
import dimensions_search_api_client as dscli
import RichContextAPI

In [20]:
api_client = RichContextAPI.connect_dimensions_api()

In [22]:
def run_exact_string_search(string,api_client):
    search_string = 'search publications in full_data for "\\"{}\\"" return publications limit 5'.format(string)
    api_response = api_client.execute_query(query_string_IN = search_string )
    return api_response

In [30]:
a = run_exact_string_search(string = search_terms[1],api_client = api_client)

In [149]:
dataset_returns = []
for d in ncses_datasets:
    ds_md = {'datasets':d['id']}
    search_terms = d['title']
    result_list = []
    for t in search_terms:
        ds_results = run_exact_string_search(string = t, api_client=api_client)
        md_results = ds_results['publications']
        [m.update(ds_md) for m in md_results]
        [m.update({'search_term':t}) for m in md_results]
        result_list.append(md_results)
#         result_list = flatten(result_list)
        dataset_returns.append(result_list)


In [158]:
dimensions_results = flatten(flatten(dataset_returns))

In [233]:
publications_md = []
final_df = pd.DataFrame()
for i,p in enumerate(dimensions_results):
    if isinstance(p,list):
        np = p[0]
    elif not isinstance(p,list):
        np = p
    p_dict = {'title':np['title'],'journal':np['journal']['title'],'dataset':np['search_term']}
    p_dict_df = p_dict
    try:
        author_list = np['author_affiliations'][0]
        author_values = []
        temp_df = pd.DataFrame()
        for a in author_list:
            the_name = a['first_name']+ ' '+a['last_name']+ ', '+a['affiliations'][0]['name']
            auth_dict = {'first_name':a['first_name'],'last_name':a['last_name'],'affiliation':a['affiliations'][0]['name']}
            author_values.append(the_name)
        auth_final = ",".join(author_values)
        p_dict_df.update({'authors':auth_final})
        p_df = pd.DataFrame(p_dict_df,index=[0])
        temp_df = temp_df.append(p_df, ignore_index=True)
            
#             p_dict.update({'authors':author_values})
#             publications_md.append(p_dict)
        final_df = final_df.append(temp_df,ignore_index = True)
    except:
        pass

In [237]:
final_df = final_df.drop_duplicates()

In [238]:
# final_df_export = final_df[['title','journal','first_name','last_name','affiliation','dataset']]
final_df.to_csv('ncses_example_linkages.csv')

In [239]:
# final_df